<a href="https://colab.research.google.com/github/BritneyMuller/colab-notebooks/blob/master/MIA_Keyword_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
from textblob import TextBlob

In [ ]:
def get_categories(searched_nouns, keywords, n_categories=3):
    """
    return n highest weighted keywords from search terms as tuple
    """

    matches = []

    for keyword in keywords.itertuples():
        if len(matches) == n_categories:
            break

        try:
            if keyword.Index in searched_nouns:
                matches.append(keyword.Index)
        except:
            continue

    missing_count = abs(len(matches) - n_categories)

    for _ in range(missing_count):
        matches.append(None)

    return matches

In [ ]:
def get_nouns(search_terms):
    """
    nltk blob tags
    NN: Noun, singular or mass
    NNS: Noun, plural
    NNP: Proper noun, singular Phrase
    NNPS: Proper noun, plural
    """

    nouns = None
    try:
        blob = TextBlob(search_terms)
        nouns = [
            noun[0]
            for noun in blob.tags
            if noun[1] in ("NN", "NNS", "NNP", "NNPS")
        ]
    except:
        pass

    return nouns

In [ ]:
def get_nouns_phrases(search_terms):
    nouns = None
    try:
        nouns = TextBlob(search_terms).noun_phrases[0]
        nouns = tuple(nouns.split())
    except:
        pass

    return nouns

In [ ]:
def main(input_file, output_file):
    df = pd.read_excel(input_file)

    # clean column names
    df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

    # df homogenize keywords column - currently mixed types
    # make all strings
    df.keywords = df.keywords.astype(str)

    # row wise -- if no nouns it will be empty
    df["nouns"] = df.apply(lambda row: get_nouns(row.keywords), axis=1)

    # duplicate record per noun for frequency count
    df_exploded = df.explode("nouns")

    # TODO: before applying weights, map different keywords that mean the same thing to the unifying keyword
    # Highest to lowest
    weighted_keywords = df_exploded.groupby(
        ["nouns"]
    ).keywords.agg(["count"]).sort_values(by=["count"], ascending=False)

    # df['cat_1'], df['cat_2'], df['cat_3'] = [None, None, None]
    # df['cat_1'], df['cat_2'], df['cat_3'] = df.apply(
    #     # lambda row, weighted_keywords=weighted_keywords: get_categories(row.keywords, weighted_keywords),
    #     lambda row: get_categories(row.keywords, weighted_keywords),
    #     axis=1
    # )
    df['results'] = df.apply(
        # lambda row, weighted_keywords=weighted_keywords: get_categories(row.keywords, weighted_keywords),
        lambda row: get_categories(row.nouns, weighted_keywords),
        axis=1
    )

    df[
        ['cat_1', 'cat_2', 'cat_3']
    ] = pd.DataFrame(df.results.values.tolist(), index=df.index)


    df.to_excel(output_file)

In [ ]:

df = pd.read_excel("/content/test.xlsx")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
df.head()

,Keywords,Volume
0,aeropress coffee vs moka pot,10.0
1,aeropress espresso grind,10.0
2,aeropress grind,390.0
3,aeropress grind level,10.0
4,aeropress grind setting,20.0


In [ ]:
# clean column names
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

# df homogenize keywords column - currently mixed types
# make all strings
df.keywords = df.keywords.astype(str)

# row wise -- if no nouns it will be empty
df["nouns"] = df.apply(lambda row: get_nouns(row.keywords), axis=1)

# duplicate record per noun for frequency count
df_exploded = df.explode("nouns")

In [ ]:
# TODO: before applying weights, map different keywords that mean the same thing to the unifying keyword
# Highest to lowest
weighted_keywords = df_exploded.groupby(
["nouns"]
).keywords.agg(["count"]).sort_values(by=["count"], ascending=False)

# df['cat_1'], df['cat_2'], df['cat_3'] = [None, None, None]
# df['cat_1'], df['cat_2'], df['cat_3'] = df.apply(
#     # lambda row, weighted_keywords=weighted_keywords: get_categories(row.keywords, weighted_keywords),
#     lambda row: get_categories(row.keywords, weighted_keywords),
#     axis=1
# )
df['results'] = df.apply(
# lambda row, weighted_keywords=weighted_keywords: get_categories(row.keywords, weighted_keywords),
lambda row: get_categories(row.nouns, weighted_keywords),
axis=1
)

df[
['cat_1', 'cat_2', 'cat_3']
] = pd.DataFrame(df.results.values.tolist(), index=df.index)

In [ ]:
df.head()

,keywords,volume,nouns,results,cat_1,cat_2,cat_3
0,aeropress coffee vs moka pot,10.0,"[coffee, vs, moka, pot]","[coffee, moka, pot]",coffee,moka,pot
1,aeropress espresso grind,10.0,"[aeropress, espresso, grind]","[espresso, grind, aeropress]",espresso,grind,aeropress
2,aeropress grind,390.0,"[aeropress, grind]","[grind, aeropress, None]",grind,aeropress,None
3,aeropress grind level,10.0,"[aeropress, grind, level]","[grind, aeropress, level]",grind,aeropress,level
4,aeropress grind setting,20.0,"[aeropress, grind]","[grind, aeropress, None]",grind,aeropress,None


In [ ]:
df.to_csv("test_output.csv",index  = False)